In [3]:
#loading libraries
suppressMessages(library("DESeq2"))
suppressMessages(library(edgeR))
suppressMessages(library(lubridate))
suppressMessages(library(dplyr))
suppressMessages(library(tidyverse))
suppressMessages(library(gsheet))
suppressMessages(library(data.table))
suppressMessages(library(vegan))
suppressMessages(library(EnvStats))
suppressMessages(library(stringr))
suppressMessages(library(vegan))

In [ ]:
# This code was used to calculate cell-types of origin's simpson diversity 

In [3]:
rm(list=ls())
sample_key = fread("/workdir/omm35/urine_plasma_cfRNA_comparison/metadata_key/plasma_urine_key.tsv")

sample_key = sample_key  %>% filter(use_pair=="TRUE")

qc_pass_samples = fread("/workdir/omm35/paper_urine_cfrna/metadata/QC_pass_samples.csv")

In [4]:
sample_key_urine = sample_key  
sample_key_plasma = sample_key



# #For urine
sample_key_urine$sample_id = sample_key_urine$sample_id_urine
sample_key_urine$event_sym = factor(sample_key_urine$event_sym)

# #For plasma
sample_key_plasma$sample_id = sample_key_plasma$sample_id_plasma
sample_key_plasma$event_sym = factor(sample_key_plasma$event_sym)
urinePlasmaMapping = sample_key_urine  %>% select(Record_ID,SubjectID,event_sym,sample_id_plasma,sample_id_urine)
colnames(urinePlasmaMapping)[3]="Event"

In [5]:
patient_metadata = fread("/workdir/omm35/urine_cfRNA_analysis/metadata/patient_metadata.tsv")
patient_metadata = patient_metadata[,c("Record_ID","age","sex","group","hct_date")]
patient_metadata$Record_ID  = as.character(patient_metadata$Record_ID)

In [6]:
calculate_simpson.diversity = function(pathtofile,samples_after_qc){
    tissues = fread(pathtofile, header=TRUE)
    tissues$sample_id=gsub("\\.",'-',tissues$sample_id)
    tissues = tissues[tissues$sample_id  %in% samples_after_qc, ]
    tissues = column_to_rownames(tissues , var = "sample_id")
    simpson_df <- as.data.frame(diversity(tissues, "simpson"))
    simpson_df = simpson_df  %>% rownames_to_column(var="sample_id")
    colnames(simpson_df)[2]="simpson_index"
    return(simpson_df)
}

In [7]:
path = "/workdir/omm35/test_cfrna_deconvolution/output/gvhd_urine_updated_ref_BP.protein_coding.tsv"
urine.simpson.df = calculate_simpson.diversity(path,sample_key_urine$sample_id)
urine.simpson.df = urine.simpson.df  %>% left_join(urinePlasmaMapping, by=c("sample_id"="sample_id_urine"))
urine.simpson.df  = urine.simpson.df  %>%  select(-sample_id_plasma)
urine.simpson.df$analyte="Urine-cfRNA"

GVHD-Plasma

In [8]:
calculate_simpson.diversity = function(pathtofile,samples_after_qc){
    tissues = fread(pathtofile, header=TRUE)
    tissues = tissues[tissues$sample_id  %in% samples_after_qc, ]
    tissues = column_to_rownames(tissues , var = "sample_id")
    simpson_df <- as.data.frame(diversity(tissues, "simpson"))
    simpson_df = simpson_df  %>% rownames_to_column(var="sample_id")
    colnames(simpson_df)[2]="simpson_index"
    return(simpson_df)
}

In [9]:
path = "/workdir/omm35/test_cfrna_deconvolution/output/gvhd_plasma_updated_ref_BP.protein_coding.tsv"
plasma.simpson.df = calculate_simpson.diversity(path,sample_key_plasma$sample_id)
plasma.simpson.df = plasma.simpson.df  %>% left_join(urinePlasmaMapping, by=c("sample_id"="sample_id_plasma"))
plasma.simpson.df  = plasma.simpson.df  %>%  select(-sample_id_urine)
plasma.simpson.df$analyte="Plasma-cfRNA"

For IRB 

In [10]:
################plasma
path_irb_plasma<-"/workdir/omm35/test_cfrna_deconvolution/healthy_plasma/healthy_plasma_updated_ref_BP.protein_coding.tsv"
# irb_tissuesPlasma$sample_id=gsub("\\.","-",irb_tissuesPlasma$sample_id)
# plasma_df = rbind(gvhd_tissuesPlasma,irb_tissuesPlasma)
# plasmadf=plasma_df

################urine 
#path_irb_urine<-"/workdir/omm35/test_cfrna_deconvolution/healthy_urine/healthy_urine_BP.protein_coding.tsv"
# irb_tissuesUrine$sample_id=gsub("\\.","-",irb_tissuesUrine$sample_id)
# urine_df = rbind(gvhd_tissuesUrine,irb_tissuesUrine)
# urinedf=urine_df
calculate_simpson.diversity = function(pathtofile,samples_after_qc){
    tissues = fread(pathtofile, header=TRUE)
    tissues$sample_id=gsub("\\.",'-',tissues$sample_id)
    tissues = tissues[tissues$sample_id  %in% samples_after_qc, ]
    tissues = column_to_rownames(tissues , var = "sample_id")
    simpson_df <- as.data.frame(diversity(tissues, "simpson"))
    simpson_df = simpson_df  %>% rownames_to_column(var="sample_id")
    colnames(simpson_df)[2]="simpson_index"
    return(simpson_df)
}

In [11]:
merged.simpson.df = rbind(urine.simpson.df,plasma.simpson.df)
merged.simpson.df = merged.simpson.df[merged.simpson.df$Event  %in% c('PR','D0','E','1M','2M','3M','6M'),]
merged.simpson.df$Event=  factor(merged.simpson.df$Event,levels=c('PR','D0','E','1M','2M','3M','6M'))

In [2]:
merged.simpson.df%>% ggplot(.,aes(Event ,simpson_index, fill= analyte))+geom_boxplot(lwd=0.5, outlier.size = 0.3)+
   geom_point(position = position_jitterdodge( jitter.width = 0.2,
  jitter.height = 0,
  dodge.width = 0.75,
  seed = NA), alpha=0.1) + scale_fill_manual(values=c('#EE6677','#4477AA'))+ theme_classic()+
  ylab("Simpson Index")+ ylim(0,1)+ theme(
    axis.title.x = element_text(size = 8,family = "Helvetica", color="black"),
    axis.text.x = element_text(size = 8,family = "Helvetica", color="black"),
    axis.title.y = element_text(size = 8,family = "Helvetica", color="black"),
    axis.text.y = element_text(size=6,family = "Helvetica", color="black"),
    #legend.position = c(0.8, 0.8),
    legend.position = "none",
    legend.text = element_text(size=6),
    legend.title = element_blank(),
    legend.box.background = element_rect(),
  )

merged.simpson.df%>% ggplot(.,aes(analyte,simpson_index, fill= analyte))+geom_boxplot(lwd=0.5, outlier.size = 0.3)+
   geom_point(position = position_jitterdodge( jitter.width = 0.2,
  jitter.height = 0,
  dodge.width = 0.75,
  seed = NA), alpha=0.1) + scale_fill_manual(values=c('#EE6677','#4477AA'))+ theme_classic()+
  ylab("Simpson Index")+
ggpubr::stat_compare_means(
comparisons= list(c("Urine-cfRNA","Plasma-cfRNA")),
method="wilcox",
size = 2.5,
method.args=list(exact=TRUE,paired=FALSE)
)+ theme(
    axis.title.x = element_text(size = 8,family = "Helvetica", color="black"),
    axis.text.x = element_text(size = 8,family = "Helvetica", color="black"),
    axis.title.y = element_text(size = 8,family = "Helvetica", color="black"),
    axis.text.y = element_text(size=6,family = "Helvetica", color="black"),
    #legend.position = c(0.8, 0.8),
    legend.position = "none",
    legend.text = element_text(size=6),
    legend.title = element_blank(),
    legend.box.background = element_rect(),
  )

In [1]:
# pairwise.wilcox.test(x=merged.simpson.df[merged.simpson.df$analyte=="Plasma-cfRNA",]$simpson_index, g=merged.simpson.df$Event, p.adjust.method = "none")
# pairwise.wilcox.test(x=merged.simpson.df[merged.simpson.df$analyte=="Urine-cfRNA",]$simpson_index, g=merged.simpson.df$Event, p.adjust.method = "none")